<a href="https://colab.research.google.com/github/F1ameX/TUSUR_ML/blob/main/TUSUR_HW_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Первый вариант



**Задача**: Использовать предобученную языковую модель для генерации продолжения текста.

В рамках задачи необходимо использовать библиотеку transformers и небольшую GPT-модель для генерации продолжения текста с заданными параметрами. Если модель сгенерировала неполное предложение (без точки/восклицательного знака/знака вопроса/многоточия), то удалить эту часть и оставить только полные предложения. Затем убрать параметр repetition_penalty и сравнить качество генерации.

## Установка и импорт необходимых библиотек

In [11]:
!pip install --quiet transformers

In [12]:
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

## Загрузка модели и токенизатора

In [13]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

## Формирование незаконченных предложений разной длины

Для эксперимента проведём анализ по трём разным незаконченным предложениям: длинной, средней и короткой длин соответственно. Идея в том, что предложения, которые практически завершены, модель будет продолжать чаще одинаково, чем в тех, где есть больше места "разгуляться".

In [14]:
long_unfinished = ['Возьму с собой зонт, потому что завтра будет плохая',
                   'Он приехал на дорогой машине, возможно, у него хорошая',
                   'Завтра я собираюсь встретиться с друзьями, потому что важно поддерживать',
                   'Вчера мы посетили новый ресторан, и я до сих пор думаю, что еда была просто',
                   'Наша команда подготовила доклад для конференции, осталось лишь']

middle_unfinished = ['Она купила новый телефон, который',
                     'Мы планируем поехать в отпуск, но',
                     'Сегодня на работе было довольно спокойно, хотя',
                     'Я пытался разобраться в этом вопросе, но',
                     'Он давно не появлялся в нашем офисе, все']

short_unfinished = ['Он сказал, что',
                    'Я думаю, что',
                    'Она не ответила на',
                    'Мне нужно сделать это',
                    'Ты не можешь объяснить']

## Вспомогательные функции

In [15]:
# Токенизация предложений для модели
def tokenize(sentences):
  return tokenizer(sentences, padding = True, truncation = True, return_tensors = 'pt')


# Генерация пар результатов с параметром 'repetition_penalty' и без него соответственно
def generate_sentences(sentences):
  penalty = model.generate(
    input_ids = tokenize(sentences)['input_ids'],
    max_length = 150,
    repetition_penalty = 1.2)

  no_penalty = model.generate(
    input_ids = tokenize(sentences)['input_ids'],
    max_length = 150)

  return penalty, no_penalty


# Препроцессинг полученных результатов модели
def preprocess_result(full_sentences):
  for sentence in full_sentences:
    result = tokenizer.decode(sentence, skip_special_tokens = True)
    result = re.split(r'[.!?]', result)[0].replace('\n', '')
    print(result)
  print('=' * 100)

## Инференс модели

Сгенерируем продолжение всех трёх видов предложений.

In [16]:
long_with_penalty, long_without_penalty = generate_sentences(long_unfinished)
middle_with_penalty, middle_without_penalty = generate_sentences(middle_unfinished)
short_with_penalty, short_without_penalty = generate_sentences(short_unfinished)

In [17]:
print('Длинные предложения с ограничением repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(long_with_penalty)
print('Длинные предложения без ограничения repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(long_without_penalty)

Длинные предложения с ограничением repetition_penalty = 1.2
Возьму с собой зонт, потому что завтра будет плохая погода
Он приехал на дорогой машине, возможно, у него хорошая машина
Завтра я собираюсь встретиться с друзьями, потому что важно поддерживать отношения
Вчера мы посетили новый ресторан, и я до сих пор думаю, что еда была просто великолепна
Наша команда подготовила доклад для конференции, осталось лишь подготовить его к печати
Длинные предложения без ограничения repetition_penalty = 1.2
Возьму с собой зонт, потому что завтра будет плохая погода
Он приехал на дорогой машине, возможно, у него хорошая машина
Завтра я собираюсь встретиться с друзьями, потому что важно поддерживать отношения с людьми, которые мне дороги
Вчера мы посетили новый ресторан, и я до сих пор думаю, что еда была просто великолепна
Наша команда подготовила доклад для конференции, осталось лишь подготовить его к печати


In [18]:
print('Средние предложения с ограничением repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(middle_with_penalty)
print('Средние предложения без ограничения repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(middle_without_penalty)

Средние предложения с ограничением repetition_penalty = 1.2
Она купила новый телефон, который ей очень нравился
Мы планируем поехать в отпуск, но не знаем с чего начать
Сегодня на работе было довольно спокойно, хотя и не очень хорошо
Я пытался разобраться в этом вопросе, но не смог
Он давно не появлялся в нашем офисе, все время пропадал где-то
Средние предложения без ограничения repetition_penalty = 1.2
Она купила новый телефон, который ей очень нравился
Мы планируем поехать в отпуск, но не знаем, как это сделать
Сегодня на работе было довольно спокойно, хотя и не очень хорошо
Я пытался разобраться в этом вопросе, но не смог
Он давно не появлялся в нашем офисе, все время пропадал в офисе, а теперь вот снова появился


In [19]:
print('Короткие предложения с ограничением repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(short_with_penalty)
print('Короткие предложения без ограничения repetition_penalty = 1.2')
print('=' * 100)
preprocess_result(short_without_penalty)

Короткие предложения с ограничением repetition_penalty = 1.2
Он сказал, что он не может быть в этом уверен
Я думаю, что это не так
Она не ответила на его вопрос
Мне нужно сделать это, — сказал он
Ты не можешь объяснить, почему ты так поступил
Короткие предложения без ограничения repetition_penalty = 1.2
Он сказал, что он не может быть в этом уверен
Я думаю, что это не так
Она не ответила на его вопрос
Мне нужно сделать это, — сказал он
Ты не можешь объяснить, почему ты не можешь быть с ним


## Вывод

Эксперимент показал, что результаты генерации предложений не сильно различались, несмотря на изменение длины исходных фраз и настройки параметра repetition_penalty. Модель продолжала генерировать тексты схожим образом независимо от того, было ли начальное предложение коротким, средним или длинным. Это может свидетельствовать о том, что модель не проявляет значительной зависимости от длины начального контекста и параметра, предназначенного для снижения повторений, при условии, что контекст достаточно ясен. Вполне возможно, что модель больше ориентируется на структуру фразы и семантические связи, чем на продолжительность начального текста.

# Второй вариант

**Задача**: Использовать предобученную языковую модель для оценки тональности текста.

В рамках задачи необходимо использовать библиотеку transformers и небольшую BERT-модель для определения степени токсичности высказывания.

## Установка и импорт необходимых библиотек

In [1]:
!pip install --quiet transformers

In [74]:
from transformers import pipeline

## Загрузка модели и токенизатора

In [75]:
pipe = pipeline("text-classification", model="s-nlp/russian_toxicity_classifier")

Device set to use cpu


## Формирование высказываний на русском языке

In [78]:
statements = [
    'Сегодня отличный день для прогулки на свежем воздухе!',
    'Погода сегодня непредсказуемая, но не так уж и плохо.',
    'Я не могу найти свои ключи, и ты вряд ли сможешь помочь, ты ведь бездарь',
    'Я горжусь своими достижениями в работе!',
    'Вчера был ужасно отстойный день, ничего особенного.',
    'Ты очень сильно раздражаешь меня, лучше заткнись!',
    'Твоя помощь ничего не значит для меня, пустоголовый!',
    'Мне нравится слушать музыку в этот вечер.',
    'Дорога была длинной, но в целом комфортной.',
    'Ты просто мега глупый',
]

## Инференс модели

In [89]:
# Словарь для замены слов на русские
trans_dict = {
    'neutral': 'нейтральное',
    'toxic': 'токсичное'
}


# Получение результатов по каждому из высказываний
for statement in statements:
  tokens = tokenizer.encode(statement, truncation = True)
  truncated_text = tokenizer.decode(tokens, skip_special_tokens = True)
  result = pipe(truncated_text)[0]
  print(f"Модель уверена на {result['score']}, что высказывание \n'{statement}'\n{trans_dict[result['label']]}\n")

Модель уверена на 0.9977224469184875, что высказывание 
'Сегодня отличный день для прогулки на свежем воздухе!'
нейтральное

Модель уверена на 0.9984690546989441, что высказывание 
'Погода сегодня непредсказуемая, но не так уж и плохо.'
нейтральное

Модель уверена на 0.8968100547790527, что высказывание 
'Я не могу найти свои ключи, и ты вряд ли сможешь помочь, ты ведь бездарь'
токсичное

Модель уверена на 0.9982578158378601, что высказывание 
'Я горжусь своими достижениями в работе!'
нейтральное

Модель уверена на 0.9890566468238831, что высказывание 
'Вчера был ужасно отстойный день, ничего особенного.'
нейтральное

Модель уверена на 0.9324531555175781, что высказывание 
'Ты очень сильно раздражаешь меня, лучше заткнись!'
токсичное

Модель уверена на 0.959291398525238, что высказывание 
'Твоя помощь ничего не значит для меня, пустоголовый!'
токсичное

Модель уверена на 0.9973582625389099, что высказывание 
'Мне нравится слушать музыку в этот вечер.'
нейтральное

Модель уверена на 0.9

## Вывод

Проверенная модель продемонстрировала хорошую способность различать положительные, нейтральные и токсичные высказывания. В ходе тестирования нейтральные комментарии не были ошибочно классифицированы как негативные. Это подтверждает, что модель не проявляет склонности оценивать нейтральные комментарии ближе к отрицательным высказываниям, что свидетельствует о корректной работе классификатора и его способности точно разделять эти категории.